In [12]:
import pandas as pd
import re
import json
from pathlib import Path
from pprint import pprint

Path.cwd()

PosixPath('/home/yjunteng/t5-for-sparql/result_notebooks')

In [13]:
result_dir = Path.cwd().parent / "models" / "longt5"

for f in result_dir.iterdir():
  if f.is_file() and f.suffix == ".json":
    print(f.name)

result_path = result_dir / "val_18.json"

val_1.json
val_18.json
val_2.json
val_12.json
val_8.json
val_19.json
val_10.json
val_13.json
val_9.json
val_14.json
val_17.json
val_11.json
val_3.json
val_16.json
val_7.json
val_15.json
val_0.json
val_5.json
val_4.json
val_6.json


In [14]:
with open(result_path) as f:
  result_json = json.load(f)

df_raw = pd.DataFrame(result_json).convert_dtypes()
print(f"Number of results: {len(df_raw)}")
print("Dtypes")
pprint(df_raw.dtypes)
df_raw.head()

Number of results: 2418
Dtypes
Ques    string
Gold    string
Gene    string
dtype: object


,Ques,Gold,Gene
0,WHERE IS THE PUBLIC OFFICE OF THE PORTRAIT IF ...,<extra_id_6> <extra_id_39> <extra_id_19> <extr...,<extra_id_6><extra_id_39><extra_id_19><extra_i...
1,What is had underlying cause of main events of...,<extra_id_6> <extra_id_39> <extra_id_19> <extr...,<extra_id_6><extra_id_39><extra_id_19><extra_i...
2,What is Robert Falcon Scott's Dreadnought Proj...,<extra_id_6> <extra_id_21> <extra_id_39> <extr...,<extra_id_6><extra_id_21><extra_id_39><extra_i...
3,What award did Yury Romanenko receive in 1976?...,<extra_id_6> <extra_id_39> <extra_id_19> <extr...,<extra_id_6><extra_id_39><extra_id_19><extra_i...
4,Was Ignazio Marino the head of government of R...,<extra_id_4> <extra_id_19> <extra_id_33> <extr...,<extra_id_4><extra_id_19><extra_id_33><extra_i...


In [15]:
df_raw["gold"] = df_raw.Gold.str.replace(" ", "")
df_raw["gene"]= df_raw.Gene.str.replace(" ","").str.replace("</s>", "").str.replace("<pad>","")

In [16]:
df_true = df_raw[df_raw.gold == df_raw.gene].loc[:,("Ques", "gold", "gene")]
df_false = df_raw[df_raw.gold != df_raw.gene].loc[:,("Ques", "gold", "gene")]

In [17]:
print(f"True rate: {len(df_true) / len(df_raw)}")
print(f"False rate: {len(df_false) / len(df_raw)}")

True rate: 0.771712158808933
False rate: 0.228287841191067


In [18]:
print(df_false.gold.iloc[1])
print(df_false.gene.iloc[1])

<extra_id_6><extra_id_21><extra_id_39><extra_id_46><extra_id_19><extra_id_33><extra_id_39><extra_id_54>p31<extra_id_53>q1328899<extra_id_38><extra_id_39><extra_id_2><extra_id_46><extra_id_38><extra_id_22><extra_id_1><extra_id_29><extra_id_1><extra_id_23><extra_id_1><extra_id_46><extra_id_12><extra_id_31><extra_id_34>t13<extra_id_34><extra_id_12><extra_id_12><extra_id_38><extra_id_22><extra_id_1><extra_id_13><extra_id_1><extra_id_46><extra_id_12><extra_id_37><extra_id_34>en<extra_id_34><extra_id_12><extra_id_15><extra_id_8>25
<extra_id_6><extra_id_21><extra_id_39><extra_id_46><extra_id_19><extra_id_33><extra_id_39><extra_id_54>p31<extra_id_53>q1328899<extra_id_38><extra_id_39><extra_id_2><extra_id_46><extra_id_38><extra_id_22><extra_id_1><extra_id_32><extra_id_1><extra_id_23><extra_id_1><extra_id_46><extra_id_12><extra_id_31><extra_id_34>t13<extra_id_34><extra_id_12><extra_id_12><extra_id_38><extra_id_22><extra_id_1><extra_id_13><extra_id_1><extra_id_46><extra_id_12><extra_id_37><extra_

In [19]:
# import requi9red module
import sys
 
# append the path of the
# parent directory
sys.path.append("..")

from src.pipeline.baselines.LCQ2Processor import BaselineMasker

masker = BaselineMasker("../src/pipeline/baselines/vocab.json")
# print(masker.vocab_unmask)

def _unmask(text):
  return text.replace(">", "> ").replace("<", " <").replace("  "," ")

for i in range(20, 33):
  print("Ques:", masker.unmask(df_false.Ques.iloc[i]))
  print("Gene:", masker.unmask(_unmask(df_false.gene.iloc[i])))
  print("Gold:", masker.unmask(_unmask(df_false.gold.iloc[i])))
  print()


Ques: As a higher taxon of Abies, name the plant known by the common name of ( script). [DEF] wd: q25350 Abies [DEF] wdt: p171 parent taxon [DEF] wdt: p1843 taxon common name
Gene: select ?vr0 where { wd: q25350 wdt: p171 ?vr1 . ?vr0 wdt: p1843 ?vr1 filter ( contains ( ?vr1 , ' <unk> ' ) ) }
Gold: select ?vr0 where { wd: q25350 wdt: p171 ?vr0 . ?vr0 wdt: p1843 ?vr1 filter ( contains ( ?vr1 , ' 松科 ' ) ) }

Ques: From what country is F. Scott Fitzgerald, who was born in St. Paul? [DEF] wd: q93354 F. Scott Fitzgerald [DEF] wd: q28848 Saint Paul [DEF] p: p19 place of birth [DEF] ps: p19 place of birth [DEF] pq: p17 country
Gene: select ?vr0 where { wd: q93354 p: p19 ?vr0 . ?vr0 ps: p19 wd: q28848 . ?vr0 pq: p17 ?vr1 }
Gold: select ?vr1 where { wd: q93354 p: p19 ?vr0 . ?vr0 ps: p19 wd: q28848 . ?vr0 pq: p17 ?vr1 }

Ques: Is schizophrenia commd10 genetically? [DEF] wd: q41112 schizophrenia [DEF] wd: q18040268 COMMD10 [DEF] p: p2293 genetic association [DEF] ps: p2293 genetic association [DEF